This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [ ]:
####################################################################################################
# Following references have been used in this project:
# - Course exercises in the Udacity Classroom
# - Official documentation for LangChain: https://python.langchain.com/v0.1/docs/modules/
####################################################################################################

In [ ]:
#############################################
# Step 1: Setting up the Python application #
#############################################

In [49]:
# LangChain version
# OpenAI version
!pip show langchain
!pip show openai

Name: langchain
Version: 0.0.305
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Name: openai
Version: 0.28.1
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 


In [51]:
# Import modules

In [52]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
import openai
import os

In [53]:
# Define your OpenAI API key 
# api_base = "https://openai.vocareum.com/v1"
# openai.api_base = api_base

api_key = ""
openai.api_key = api_key

# NB: I tried using the Vocareum key recently provided by Udacity, however, this lead to errors saying 
# that I had an insufficient budget available. There seems to be problem with proxy_server (parameter open_api_base)
# Instead, I have defined an environment variable 'OPENAI_API_KEY' with my own key from OpenAI, which still has funds on it, 
# and I have used command: llm = ChatOpenAI() instead of command: llm = OpenAI(...)
# This works fine! 
# See step 3 below.

In [54]:
###########################################
# Step 2: Generating real estate listings #
###########################################

In [55]:
# Generate real estate listings using a Large Language Model (LLM). Generate at least 10 listings.

In [56]:
specifications = [
    {'neighborhood':'Brixton', 'price': 225000, 'bedroom': 1, 'bathroom': 1, 'size': 850 },
    {'neighborhood':'Wimbledon', 'price': 320000, 'bedroom': 2, 'bathroom': 1, 'size': 1050 },
    {'neighborhood':'Twickenham', 'price': 450000, 'bedroom': 3, 'bathroom': 2, 'size': 1550 },
    {'neighborhood':'Canary Wharf', 'price': 320000, 'bedroom': 2, 'bathroom': 2, 'size': 1250 },
    {'neighborhood':'Hammersmith', 'price': 390000, 'bedroom': 3, 'bathroom': 1, 'size': 1450 },
    {'neighborhood':'Camden Town', 'price': 295000, 'bedroom': 1, 'bathroom': 1, 'size': 975 },
    {'neighborhood':'Richmond', 'price': 505000, 'bedroom': 3, 'bathroom': 2, 'size': 1750 },
    {'neighborhood':'Notting Hill', 'price': 650000, 'bedroom': 4, 'bathroom': 2, 'size': 2025 },
    {'neighborhood':'Covent Garden', 'price': 425000, 'bedroom': 2, 'bathroom': 1, 'size': 1250 },
    {'neighborhood':'Ealing', 'price': 350000, 'bedroom': 2, 'bathroom': 1, 'size': 1150 }    
]

In [57]:
prompt_template = """ 
Generate a real estate listing. The listing should contain neighborhood, price, number of bedrooms, number of
of bathrooms and house size (in square feet). The listing should contain a description and a neighborhood
description.

The real estate listing should use following values

Neighborhood: {neighborhood}
Price: £{price}
Bedrooms: {bedroom}
Bathrooms: {bathroom}
House Size: {size} sqft

An example of a real estate listing is provided below:

Neighborhood: Green Oaks
Price: £600,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. 
Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. 
Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""


In [58]:
# Call the OpenAI API with your prompt and print the response
# Function to call the OpenAI GPT-3.5 API
def generate_real_estate_listing(prompt_template):
    try:
        # Calling the OpenAI API with a system message and our prompt in the user message content
        # Use openai.ChatCompletion.create for openai < 1.0
        # openai.chat.completions.create for openai > 1.0
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": "You are a real estate agent. You are writing about real estate listing. "
          },
          {
            "role": "user",
            "content": prompt_template
          }
          ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
        # The response is a JSON object containing more information than the generated review. We want to return only the message content
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

In [59]:
# Generating the response from the model
listings = []
count = 0
for s in specifications:
    prompt = prompt_template.format(neighborhood=s['neighborhood'], price=s['price'], bedroom=s['bedroom'], bathroom=s['bathroom'], size=s['size'])
    listing = generate_real_estate_listing(prompt)
    listings.append("\n" + listing + "\n")
    file1 = open(f'data/listing{count}.txt', 'w')
    file1.write(listing)
    file1.close()
    count += 1

In [60]:
# Each listing is stored in a separate file. To be used with class DirectoryLoader further down. 
sorted(os.listdir('data'))

['listing0.txt',
 'listing1.txt',
 'listing2.txt',
 'listing3.txt',
 'listing4.txt',
 'listing5.txt',
 'listing6.txt',
 'listing7.txt',
 'listing8.txt',
 'listing9.txt']

In [61]:
# Also, create a file called 'listings' that contain all the listings together  
file2 = open('Listings.txt', 'w')
file2.writelines(listings)
file2.close()

In [62]:
################################################
# Step 3: Store listings in a vector database  #
################################################

In [63]:
# import os
os.environ['OPENAI_API_KEY'] = api_key

In [64]:
# model_name = 'gpt-3.5-turbo'
# llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000, openai_api_key=api_key, openai_api_base=api_base)
# This has been replaced with the next line of code
llm = ChatOpenAI()

In [65]:
# Following package was missing and had to be installed
# !pip install unstructured

In [66]:
# Create an instance of the DirectoryLoader class based on the listings stored in the 'data' directory
loader = DirectoryLoader('./data', glob="**/*.txt", loader_cls=TextLoader)
docs = loader.load()

In [67]:
len(docs)

10

In [68]:
print(docs[0])

page_content="Neighborhood: Richmond\nPrice: £505,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 1,750 sqft\n\nDescription: Step into luxury in this contemporary 3-bedroom, 2-bathroom home located in the sought-after neighborhood of Richmond. This stunning residence features modern finishes throughout, including sleek hardwood floors, granite countertops, and stainless steel appliances in the chef's kitchen. The spacious living room is perfect for entertaining, while the primary bedroom offers a private oasis with an ensuite bathroom. Enjoy the convenience of a two-car garage and a beautifully landscaped backyard, ideal for outdoor gatherings. This stylish and chic home is perfect for those seeking comfort and elegance in a prime location.\n\nNeighborhood Description: Richmond is a vibrant and family-friendly neighborhood known for its tree-lined streets, local cafes, and boutique shops. Residents can enjoy easy access to parks, schools, and community events, making it an ideal place to c

In [69]:
# Use class CharacterTextSplitter to split the documents into chunks.
# This stores the embeddings in a more effective way.
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [70]:
# Initialize the embeddings model
embeddings = OpenAIEmbeddings()

In [71]:
# Populate the Chroma vector database with the chunks
db = Chroma.from_documents(split_docs, embeddings)

In [72]:
##################################################
# Step 4: Building the user preference interface #
##################################################

In [73]:
# List of questions viewed as most helpful for the AI recommender
personal_questions = ["How big do you want your house to be?", 
             "What are 3 most important things for you in choosing this property?", 
             "Which amenities would you like?", 
             "Which transportation options are important to you?",
             "How urban do you want your neighborhood to be?"]
    
# List of your answers to the questions above
personal_answers = ["A comfortable 2-bedroom, 1-bathroom home with a spacious kitchen and a cozy living room.",
           "A quiet neighborhood and good local schools.",
           "A backyard for gardening and barbecues.",
           "Easy access to public transport.",
           "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

In [74]:
# Create a query based on the personal questions and answers
query = f"""
    {personal_questions[0]} {personal_answers[0]} 
    {personal_questions[1]} {personal_answers[1]}
    {personal_questions[2]} {personal_answers[2]}
    {personal_questions[3]} {personal_answers[3]}
    {personal_questions[4]} {personal_answers[4]}
    """

In [75]:
##########################################
# Step 5: Searching based on preferences #
##########################################

In [79]:
# Find top 5 semantically similar documents to the query
results = db.similarity_search(query, k=5)
print(results)

[Document(page_content='Description: Step into this elegant 2-bedroom, 1-bathroom home located in the prestigious neighborhood of Wimbledon. This charming residence exudes classic character with modern upgrades, perfect for those seeking a blend of tradition and contemporary living. The spacious living area with a cozy fireplace opens up to a gourmet kitchen equipped with stainless steel appliances and granite countertops.\n \nThe master bedroom features ample closet space and direct access to a private outdoor patio, ideal for enjoying your morning coffee or winding down in the evening. The second bedroom offers versatility as a guest room, home office, or nursery. With hardwood floors throughout and ample natural light, this home radiates warmth and comfort.\n \nStep outside to the beautifully landscaped garden, complete with a tranquil pond and lush greenery, creating a serene retreat right at your doorstep. Conveniently located near local shops, fine dining restaurants, and top-rat

In [80]:
# Query your LLM with the query and the top 5 documents
use_chain_helper = True
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    # similar_docs = db.similarity_search(query, k=5)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=results, query = query))

Based on the preferences you've outlined, the property in Wimbledon seems to align well with your criteria. It offers a comfortable 2-bedroom, 1-bathroom home with a spacious kitchen and a cozy living room. The neighborhood of Wimbledon is prestigious, providing a quiet environment. Additionally, it is conveniently located near top-rated schools. The property also features a beautifully landscaped garden with a tranquil pond, perfect for gardening and barbecues. While Wimbledon is a more suburban area, it still offers easy access to urban amenities like local shops and fine dining restaurants.


In [81]:
##############################################
# Step 6: Personalize listing descriptions #
##############################################

In [82]:
history = ChatMessageHistory()
history.add_user_message(f"""You are an AI that will provide a real estate listing based on answers to personal questions. Ask user {len(personal_questions)} questions""")
# Add questions and answers to the history
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])    

In [83]:
# Create a memory that will have a summary of the recommendations
max_rating = 100

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary", 
    input_key="input",
    buffer=f"The human answered {len(personal_questions)} personal questions). Use them to augment the real estate listing, tailoring it to resonate with the buyer’s specific preferences.",
    return_messages=True)

class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
    
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)

# Combined
memory = CombinedMemory(memories=[conversational_memory, summary_memory])

In [84]:
RECOMMENDER_TEMPLATE = """
The following is a friendly conversation between a human and an AI real estate agent. The AI follows human instructions
and provides an augmented real estate listing for a human based on the real estate listing provided. 

Summary of Recommendations:
{recommendation_summary}
Personal Questions and Answers:
{questions_and_answers}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["recommendation_summary", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
# Create a recommendation conversation chain that will let us ask AI for each retrieved listing
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)

In [85]:
# max_rating = 100
count = 1
for listing in listings:
    print("-----------------")
    print("-----------------")
    print("Listing number: " + str(count)) 
    print("-----------------")
    print("-----------------")
    listing_instructions = f"""
        =====================================
        === START REAL ESTATE LISTING ===
        {listing}
        === END REAL ESTATE LISTING ===
        =====================================
        
        LISTING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a real estate listing based only on the listing provided and human answers to questions included with the context. 
        AI should be very sensible to human personal preferences captured in the answers to personal questions and should not be influenced 
        by anything else. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for. 
        Ensure that the augmentation process enhances the appeal of the real estate listing without altering factual information.
        AI will also build a persona for human based on human answers to questions.
        OUTPUT FORMAT:
        It is very important to follow that same format as the real estate listing.
        Add the persona you came up with to the end of the real estate listing. Describe the persona in a few sentences.
        Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
        FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO UNDERSTAND YOUR REVIEW.
    """
    # Run the the recommendation chain to get a rating for the movie that will be summarized in the conversation summary
    prediction = recommender.predict(input=listing_instructions)
    print(prediction)
    count += 1

-----------------
-----------------
Listing number: 1
-----------------
-----------------


> Entering new ConversationChain chain...
Prompt after formatting:

The following is a friendly conversation between a human and an AI real estate agent. The AI follows human instructions
and provides an augmented real estate listing for a human based on the real estate listing provided. 

Summary of Recommendations:
[SystemMessage(content='The human answered 5 personal questions). Use them to augment the real estate listing, tailoring it to resonate with the buyer’s specific preferences.')]
Personal Questions and Answers:
Human: You are an AI that will provide a real estate listing based on answers to personal questions. Ask user 5 questions
AI: How big do you want your house to be?
Human: A comfortable 2-bedroom, 1-bathroom home with a spacious kitchen and a cozy living room.
AI: What are 3 most important things for you in choosing this property?
Human: A quiet neighborhood and good local scho


> Finished chain.
Neighborhood: Wimbledon
Price: £320,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,050 sqft

Description: Step into this charming 2-bedroom, 1-bathroom home located in the desirable neighborhood of Wimbledon. This cozy abode features a spacious living area, a well-appointed kitchen with modern appliances, and a bright dining space overlooking the backyard. The bedrooms offer ample natural light and closet space, providing a comfortable sanctuary for relaxation. Enjoy the convenience of an attached garage and a private backyard, perfect for outdoor entertaining or gardening. This home is ideal for those seeking a peaceful retreat in a vibrant community.

Neighborhood Description: Wimbledon is a picturesque neighborhood known for its tree-lined streets, friendly neighbors, and community events. Residents enjoy easy access to local cafes, boutique shops, and recreational facilities. Take a leisurely stroll to the nearby Wimbledon Park for a picnic or head to the tennis cour


> Finished chain.
Neighborhood: Twickenham
Price: £450,000
Bedrooms: 3
Bathrooms: 2
House Size: 1,550 sqft

Description: Step into this elegant 3-bedroom, 2-bathroom home located in the sought-after neighborhood of Twickenham. With a spacious open floor plan, this property offers a seamless flow between the living room, dining area, and modern kitchen with stainless steel appliances. The master bedroom features an ensuite bathroom with a luxurious soaking tub, while the other bedrooms are bright and inviting. Outside, the backyard provides a serene retreat with a patio area for outdoor dining and entertaining. Perfect for relaxing nights in and hosting gatherings with friends and family, this home combines charm and functionality for a comfortable lifestyle.

Neighborhood Description: Twickenham is a historic and vibrant neighborhood known for its tree-lined streets, historic architecture, and close-knit community. Residents enjoy easy access to local parks, trendy cafes, boutique sho


> Finished chain.
Neighborhood: Canary Wharf
Price: £320,000
Bedrooms: 2
Bathrooms: 2
House Size: 1,250 sqft

Description: Step into luxury living in the heart of Canary Wharf with this stunning 2-bedroom, 2-bathroom home offering breathtaking views of the waterfront. This modern residence features an open-concept floor plan with sleek finishes and floor-to-ceiling windows that flood the space with natural light. The gourmet kitchen is equipped with high-end appliances and a breakfast bar, perfect for entertaining guests. The spacious bedrooms provide a relaxing retreat, while the bathrooms are elegantly designed with contemporary fixtures. Enjoy a morning coffee on your private balcony overlooking the bustling city below. Experience the epitome of urban living in this chic Canary Wharf residence.

Neighborhood Description: Canary Wharf is a vibrant financial district situated along the Thames River, known for its iconic skyscrapers, upscale restaurants, and designer shops. Take a lei


> Finished chain.
Neighborhood: Hammersmith
Price: £390,000
Bedrooms: 3
Bathrooms: 1
House Size: 1,450 sqft

Description: Step into this charming 3-bedroom, 1-bathroom home located in the desirable neighborhood of Hammersmith. This cozy residence features a spacious living area with large windows that flood the space with natural light. The modern kitchen is perfect for whipping up delicious meals and opens up to a lovely dining area. The bedrooms offer ample space and comfortable living, making it ideal for families or individuals seeking a peaceful retreat. With a cozy backyard, perfect for summer BBQs and gatherings, this home is a true gem in Hammersmith.

Neighborhood Description: Hammersmith is a vibrant and diverse neighborhood that offers a mix of urban conveniences and a charming residential atmosphere. Enjoy strolls along tree-lined streets, visit local cafes, or explore nearby parks. The neighborhood is also known for its excellent schools, trendy boutiques, and a variety o


> Finished chain.
Neighborhood: Camden Town
Price: £295,000
Bedrooms: 1
Bathrooms: 1
House Size: 975 sqft

Description: Step into this vibrant and lively 1-bedroom, 1-bathroom urban sanctuary located in the heart of Camden Town. This stylishly appointed home features modern fixtures and fittings, enriching the open layout with a sense of sophistication. The expansive windows illuminate the space throughout the day, creating a warm and welcoming atmosphere. The cozy bedroom offers a peaceful retreat, while the sleek bathroom provides a touch of luxury. Enjoy the convenience of a contemporary kitchen equipped with top-of-the-line appliances.

Neighborhood Description: Camden Town is a trendy and eclectic neighborhood known for its artistic flair, bustling markets, and vibrant music scene. Located in the heart of London, Camden Town offers an array of eclectic boutiques, music venues, and diverse eateries. Stroll along the picturesque Regent's Canal or explore the iconic Camden Market fi


> Finished chain.
Neighborhood: Richmond
Price: £505,000
Bedrooms: 3
Bathrooms: 2
House Size: 1,750 sqft

Description: Step into luxury in this contemporary 3-bedroom, 2-bathroom home located in the sought-after neighborhood of Richmond. This stunning residence features modern finishes throughout, including sleek hardwood floors, granite countertops, and stainless steel appliances in the chef's kitchen. The spacious living room is perfect for entertaining, while the primary bedroom offers a private oasis with an ensuite bathroom. Enjoy the convenience of a two-car garage and a beautifully landscaped backyard, ideal for outdoor gatherings. This stylish and chic home is perfect for those seeking comfort and elegance in a prime location.

Neighborhood Description: Richmond is a vibrant and family-friendly neighborhood known for its tree-lined streets, local cafes, and boutique shops. Residents can enjoy easy access to parks, schools, and community events, making it an ideal place to call


> Finished chain.
Neighborhood: Notting Hill
Price: £650,000
Bedrooms: 4
Bathrooms: 2
House Size: 2,025 sqft

Description: Step into luxury living in this stunning 4-bedroom, 2-bathroom home located in the desirable neighborhood of Notting Hill. Boasting over 2,000 square feet of living space, this elegant residence offers the perfect blend of modern amenities and classic charm. The spacious bedrooms provide ample space for relaxation, while the updated bathrooms feature sleek fixtures and spa-like finishes. The kitchen is a chef's dream with high-end appliances, custom cabinetry, and quartz countertops. Entertain guests in the expansive living room with its cozy fireplace and large windows that flood the space with natural light. Step outside to the beautifully landscaped backyard, perfect for hosting summer BBQs or enjoying a morning coffee.

Neighborhood Description: Notting Hill is a vibrant and eclectic neighborhood known for its trendy shops, gourmet restaurants, and lively stre


> Finished chain.
Neighborhood: Covent Garden
Price: £425,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,250 sqft

Description: Step into this elegant 2-bedroom, 1-bathroom home situated in the bustling neighborhood of Covent Garden. The meticulously maintained property features a spacious open living area, perfect for entertaining guests or relaxing after a long day. The modern kitchen is equipped with sleek appliances and ample storage space. Both bedrooms offer generous closet space, and the full bathroom is adorned with contemporary fixtures. Enjoy the convenience of in-unit laundry and the abundance of natural light that fills the interior space. This cozy residence is a perfect blend of comfort and style, offering the ideal urban retreat in the heart of Covent Garden.

Neighborhood Description: Covent Garden is a vibrant district in central London, known for its iconic market, street performers, and cultural attractions. Residents can explore a variety of charming cafes, boutique sh


> Finished chain.
Neighborhood: Ealing
Price: £350,000
Bedrooms: 2
Bathrooms: 1
House Size: 1,150 sqft

Description: Step into this cozy 2-bedroom, 1-bathroom retreat in the desirable neighborhood of Ealing. This charming home features a bright and airy living room, perfect for relaxing or entertaining guests. The updated kitchen showcases modern appliances and ample storage space for all your culinary needs. Both bedrooms offer comfort and tranquility, with large windows letting in natural light throughout the day. The spacious backyard is a private oasis, ideal for hosting summer barbecues or enjoying a quiet evening under the stars. Don't miss the opportunity to make this delightful Ealing residence your own!

Neighborhood Description: Ealing is a vibrant neighborhood renowned for its tree-lined streets and family-friendly atmosphere. Residents of Ealing enjoy easy access to local shops, cafes, and parks, making it an ideal location for both young professionals and families. Explor